<a href="https://colab.research.google.com/github/pgordin/OptDisc2024/blob/main/Grafy3_class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from random import random
from copy import deepcopy

In [24]:
class Graph:
    def __init__(self, graph=None):
        if graph is None:
            graph = {}
        self.graph = graph

    # dict initializer
    @classmethod
    def from_dict(cls, graph):
        return cls(graph)

    # array initializer
    @classmethod
    def from_array(cls, graph: np.array, nodes: list = None):
        if nodes is None:
            nodes = [*range(1, len(graph) + 1)]
        return cls.from_dict(
            cls._array_to_dict(graph, nodes)
        )

    @staticmethod
    def from_edges(filename: str, directed = 0):
        """
        Generates a graphs from a text file, where each line defines one edge.\n
        Filename is a file path.
        """
        graph = Graph()
        file = open(filename, "r")
        for line in file:
            words = line.strip().split()
            if len(words) == 1:
                graph.add_node(words[0])
            elif len(words) > 1: # more than two words -- we use first two
                if directed:
                    graph.add_arc([words[0], words[1]])
                else:
                    graph.add_edge([words[0], words[1]])
        file.close()
        return graph

    @staticmethod
    def random_graph(nodes_num: int, prob: float):
        """
        Generates a random graph provided a number of nodes and probability of generating an edge.
        """
        rand_graph = Graph()
        for i in range(1, nodes_num + 1):
            rand_graph.add_node(i)
            for j in range(1, i):
                if random() < prob:
                    rand_graph.add_edge([i, j])
        return rand_graph

    @staticmethod
    def cycle(nodes_num: int):
        """
        Generates a cycle provided a number of nodes.
        """
        cycle = Graph()
        nodes = [*range(1, nodes_num + 1)]
        for i in nodes:
            cycle.add_edge([nodes[i-2], nodes[i-1]])
        cycle.graph = dict(sorted(cycle.graph.items()))
        return cycle

    def to_neighbourlist(self, filename: str):
        """
        Saves a graphs to a text file as a neighbour dict.\n
        Filename is a file path.
        """
        file = open(filename, "w")
        file.write(str(self))
        file.close()

    def nodes(self) -> list:
        """
        Returns list of nodes of a graph.
        """
        return [*self.graph.keys()]

    def array(self) -> np.array:
        """
        Returns the graph in array form.
        """
        return self._dict_to_array(self.graph)

    # redefinition of print for objects of class Graph
    def __str__(self):
        res = ""
        for v in self.graph:
            res += f"{v}:"
            for u in self.graph[v]:
                res += f" {u}"
            res += "\n"
        return res

    def add_node(self, node):
        """
        Adds a node to a graph.
        """
        if node not in self.graph:
            self.graph[node] = []

    def del_node(self, node):
        """
        Recursively removes a node from a graph.
        """
        if node in self.graph:
            self.graph.pop(node)
            for key in [*self.graph.keys()]:
                if node in self.graph[key]:
                    self.graph[key].remove(node)

    def add_arc(self, arc: list):
        """
        Adds arc to a graph provided a list of nodes.
        """
        u, v = arc
        self.add_node(u)
        self.add_node(v)
        if v not in self.graph[u]:
            self.graph[u].append(v)

    def add_edge(self, edge: list):
        """
        Adds edge to a graph provided a list of nodes.
        """
        u, v = edge
        if u == v:
            raise ValueError("Pętla!")
        self.add_node(u)
        self.add_node(v)
        if v not in self.graph[u]:
            self.graph[u].append(v)
        if u not in self.graph[v]:
            self.graph[v].append(u)

    def _array_to_dict(arr: np.array, nodes: list) -> dict:
        """
        Converts a graph in array form to a graph in dict form.
        """
        res_dict = {}
        for i, node in enumerate(nodes):
            neighbours = [nodes[j] for j, edge in enumerate(arr[i]) if edge]
            res_dict[node] = neighbours
        return res_dict

    def _dict_to_array(self, _dict: dict) -> np.array:
        """
        Converts a graph in dict form to a graph in array form.
        """
        n = len(_dict)
        nodes = [*_dict.keys()]
        res_arr = np.zeros(shape = (n, n), dtype=int)
        for u,v in [
            (nodes.index(u), nodes.index(v))
            for u, row in _dict.items() for v in row
        ]:
            res_arr[u][v] += 1
        return res_arr

    def Prufer(self):
      """
      Kod Prufera drzewa - zwrócony jako napis
      Wymagane, aby graf był drzewem.
      Zwraca pusty napis dla drzew o mniej niż 3 wierzchołkach.
      """
      tr = deepcopy(self.graph)   # będziemy psuć graf
      code = ""
      for i in range(len(self.graph) - 2):
        for x in sorted(tr):    # po kolei przeglądam nieusunięte wierzchołki
          if len(tr[x]) == 1:   # najmniejszy liść
            break
        v = tr[x][0]  # sąsiad najmniejszego x
        code = code + f"{v} "
        tr[v].remove(x)   # usuwam x z listy sąsiadów v
        tr.pop(x)         # usuwam x z drzewa
      return code.strip()

    @staticmethod
    def tree_from_Prufer(code: str):
        """
        Tworzy drzewo na podstawie kodu Prufera.
        """
        tree = Graph()
        clist = [int(x) for x in code.strip().split()]   # kod zamieniamy na listę liczb
        n = len(clist) + 2    # liczba wierzchołków
        vert = [v for v in range(1, n+1)]  # lista liczb od 1 do n
        for v in vert:
          tree.add_node(v)
        for i in range(n-2):
          for x in vert:
            if not x in clist:    # najmniejszy liść
              break
          v = clist.pop(0)    # usuwam pierwszy element listy (sąsiad x)
          tree.add_edge((x, v))
          vert.remove(x)
        tree.add_edge(vert)
        return tree

    def ConnectedComponents(self):
      """
      Znajduje spójne składowe w grafie nieskierowanym
      Jako wynik zwraca listę zbiorów wierzchołków
      Uwaga: jako pierwszy element listy uzyskamy zbiór wszystkich wierzchołków grafu
      """
      def DFS(u):
        """
        Przeszukiwanie w głąb
        """
        for w in self.graph[u]:
          if not w in VT[0]:  # w jeszcze nie odwiedzony
            VT[0].add(w)      # już odwiedzony
            VT[-1].add(w)     # w ostatniej spójnej składowej
            DFS(w)
      """
      VT - lista zbiorów VT[i] dla i > 0 - lista wierzchołków spójnych składowych
      dla i = 0 - lista wszystkich odwiedzonych wierzchołków
      """
      VT = [set([])]
      for v in self.graph:
        if v not in VT[0]:
          VT[0].add(v)
          VT.append(set([v]))   # zaczątek nowej spójnej składowej
          DFS(v)
      return VT

# Przykłady wykorzystania

In [3]:
graph = Graph.random_graph(10, 1/5)
print(graph)
print("---------------------------")
print(graph.ConnectedComponents())


6: 2 5 8
2: 6
5: 6
7: 1
1: 7 8
8: 1 4 6
4: 8
10: 9
9: 10

---------------------------
[{1, 2, 4, 5, 6, 7, 8, 9, 10}, {1, 2, 4, 5, 6, 7, 8}, {9, 10}]


In [4]:
graph.add_edge((11, 12))
print(graph.ConnectedComponents())


[{1, 2, 4, 5, 6, 7, 8, 9, 10, 11, 12}, {1, 2, 4, 5, 6, 7, 8}, {9, 10}, {11, 12}]


In [5]:
tree = Graph.from_dict(
    {
        1: [2],
        2: [1, 3, 4],
        3: [2],
        4: [2]
    }
)
print(tree.Prufer())

2 2


In [6]:
print(Graph.tree_from_Prufer("2 2"))

1: 2
2: 1 3 4
3: 2
4: 2



In [7]:
tree2 = Graph.tree_from_Prufer("1 2 7 3 4")
print(tree2)
print("---------------------")
print(tree2.Prufer())

1: 5 2
2: 1 7
3: 6 4
4: 3 7
5: 1
6: 3
7: 2 4

---------------------
1 2 7 3 4


In [8]:
tree3 = Graph.tree_from_Prufer("1 1 1 1 1 1 1 1")
print(tree3)
print("---------------------")
print(tree3.Prufer())

1: 2 3 4 5 6 7 8 9 10
2: 1
3: 1
4: 1
5: 1
6: 1
7: 1
8: 1
9: 1
10: 1

---------------------
1 1 1 1 1 1 1 1


In [9]:
graph = Graph.from_dict(
    {
        'a': ['b', 'c'],
        'b': ['a', 'd', 'e'],
        'c': ['a', 'f'],
        'd': ['b', 'f'],
        'e': ['b'],
        'f': ['c', 'd'],
        'g': [],
    }
)

graph_array = graph.array()
graph_nodes = graph.nodes()

print(graph)
print(graph_array)
print(graph_nodes)

a: b c
b: a d e
c: a f
d: b f
e: b
f: c d
g:

[[0 1 1 0 0 0 0]
 [1 0 0 1 1 0 0]
 [1 0 0 0 0 1 0]
 [0 1 0 0 0 1 0]
 [0 1 0 0 0 0 0]
 [0 0 1 1 0 0 0]
 [0 0 0 0 0 0 0]]
['a', 'b', 'c', 'd', 'e', 'f', 'g']


In [10]:
random_graph = Graph.random_graph(10, 1/3)
print(random_graph)

2: 1 3 4 6
1: 2 3 8 10
3: 1 2 4 8 9
4: 2 3 5
5: 4 8
6: 2 7 9 10
7: 6 9 10
8: 1 3 5
9: 3 6 7
10: 1 6 7



In [11]:
cycle = Graph.cycle(10)
print(cycle)

1: 10 2
2: 1 3
3: 2 4
4: 3 5
5: 4 6
6: 5 7
7: 6 8
8: 7 9
9: 8 10
10: 1 9



In [12]:
%%writefile lista.txt
A B
B C
B D
D C
E
F

Writing lista.txt


In [13]:
file_graph = Graph.from_edges("./lista.txt")
print(file_graph)

A: B
B: A C D
C: B D
D: B C
E:
F:



In [14]:
file_graph.add_edge(["E", "F"])
file_graph.to_neighbourlist("lista1.txt")

In [15]:
%cat lista1.txt

A: B
B: A C D
C: B D
D: B C
E: F
F: E


In [22]:
!wget https://raw.githubusercontent.com/pgordin/OptDisc2024/main/ubranie.txt

--2024-04-21 22:44:43--  https://raw.githubusercontent.com/pgordin/OptDisc2024/main/ubranie.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 186 [text/plain]
Saving to: ‘ubranie.txt’

ubranie.txt         100%[===================>]     186  --.-KB/s    in 0s      

2024-04-21 22:44:43 (11.9 MB/s) - ‘ubranie.txt’ saved [186/186]



In [23]:
ubranie_graph = Graph.from_edges("ubranie.txt", directed = 1)
print(ubranie_graph)

slipki: kalesony
kalesony: spodnie
spodnie: buty szelki
buty:
szelki: marynarka
skarpety: buty
koszula: szelki marynarka krawat
marynarka: plaszcz
krawat: marynarka
plaszcz:

